# WSDM Cup 2023: [Pre-training for Web Search](https://aistudio.baidu.com/aistudio/competition/detail/536/0/introduction) Demo Code 
## Task Description
- Pre-train a PLM with the Large Scale Web Search Session Data and finetune the PLM with the Expert Annotation Dataset.

- The award will only be honored to PaddlePaddle submission.

- The award will be honored to the teams with the code that can reproduce their results.

- Evaluation Metric is DCG@10.

More detail about the dataset analysis and baseline methods are introduced in [A Large Scale Search Dataset for Unbiased Learning to Rank](https://arxiv.org/abs/2207.03051).

## Running Demo

The following code is a fast startup.


### Step 1: Download the demo running code from github

In [ ]:
# clear the dir
!rm -rf ./*
# clone demo code
!git clone https://github.com/ChuXiaokai/WSDMCUP_BaiduPLM_Paddle.git

Cloning into 'WSDMCUP_BaiduPLM_Paddle'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 84 (delta 40), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (84/84), done.
Checking connectivity... done.


### Step 2: Download & Prepare the pre-training & evaluation dataset
Notice that the dataset are a small subset of the whole dataset for fast running the demo code.

More information about the dataset is available at https://searchscience.baidu.com/dataset_ultr.html

In [7]:
# Notice that the dataset are a small subset of the whole dataset for fast running the demo code. 
# Download train dataset
!wget https://searchscience.baidu.com/baidu_ultr/train_click/part-00000.gz -O part-00000.gz 
!wget https://searchscience.baidu.com/baidu_ultr/train_click/part-00001.gz -O part-00001.gz 

# preparing the training data
!mkdir ./WSDMCUP_BaiduPLM_Paddle/data/train_data/
!mv part-00001.gz ./WSDMCUP_BaiduPLM_Paddle/data/train_data/
# preparing the click test data, suggesting using a larger click dataset 
!zcat ./part-00000.gz|head -1000 > ./WSDMCUP_BaiduPLM_Paddle/data/click_data/test_click.data && gzip ./WSDMCUP_BaiduPLM_Paddle/data/click_data/test_click.data
!rm part-00000.gz

# preparing the validation dataset
!wget https://searchscience.baidu.com/baidu_ultr/labeled_dataset/test_data.txt -O test_data.txt 
!tail -10000 test_data.txt > valid_data.txt
!head -387572 test_data.txt > finetune.txt 
!mv valid_data.txt ./WSDMCUP_BaiduPLM_Paddle/data/annotate_data/
!mv finetune.txt ./WSDMCUP_BaiduPLM_Paddle/data/annotate_data/

--2023-01-11 18:08:15--  https://searchscience.baidu.com/baidu_ultr/train_click/part-00000.gz
正在解析主机 searchscience.baidu.com (searchscience.baidu.com)... 180.101.49.82
正在连接 searchscience.baidu.com (searchscience.baidu.com)|180.101.49.82|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 622792845 (594M) [application/octet-stream]
正在保存至: “part-00000.gz”

part-00000.gz        48%[========>           ] 290.59M  5.33MB/s    in 60s     

2023-01-11 18:09:15 (4.82 MB/s) - 在 304703712 字节处连接关闭。 重试中。

--2023-01-11 18:09:16--  (尝试次数： 2)  https://searchscience.baidu.com/baidu_ultr/train_click/part-00000.gz
正在连接 searchscience.baidu.com (searchscience.baidu.com)|180.101.49.82|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 206 Partial Content
长度： 622792845 (594M)，剩余 318089133 (303M) [application/octet-stream]
正在保存至: “part-00000.gz”

part-00000.gz        86%[+++++++++=======>   ] 514.63M  2.63MB/s    in 61s     

2023-01-11 18:10:17 (3.70 MB/s) - 在 539627968 字节处连接关闭。 重试中。

--2023-01-11 18:10:19--  (尝试次数： 3)  https://

### Step 3: Prepare the pre-trained language model for fast startup
A pre-trained model is available from the github "https://github.com/ChuXiaokai/WSDMCUP_BaiduPLM_Paddle.git"

12 Layer Model: https://drive.google.com/file/d/1dSSA3RotV4CNG5WMdAQ4ZulipjXSPwD1/view?usp=sharing

6 Layer Model: https://drive.google.com/file/d/1tcQ-_VkKbV4ESE2Wu9FgAx9wVvO6Fc7w/view?usp=sharing

3 Layer Model: https://drive.google.com/file/d/1tcQ-_VkKbV4ESE2Wu9FgAx9wVvO6Fc7w/view?usp=sharing

**Since we cannot access the google drive from here, we suggest to download the model by yourself.**

In [ ]:
# # Download the three layer model as the initialize model
# !pip install gdown
# !gdown https://drive.google.com/uc?id=1tcQ-_VkKbV4ESE2Wu9FgAx9wVvO6Fc7w
!mkdir ./WSDMCUP_BaiduPLM_Paddle/trained_model/

### Step 4: Pre-training Learning for Web Search
Training a pre-trained language model for web search.

Taking the mlm + ctr prediction as the example: 

**Set the init_parameters to the path of downloaded pre-trained model.**

In [ ]:
# finetune the model
# load the model from pretrain (in work/save_model/* )
!cd WSDMCUP_BaiduPLM_Paddle && python -u pretrain.py --init_parameters '/home/aistudio/data/data186340/save_steps70000_0.98254_1.417266.model' --emb_dim 768 --nlayer 3 --nhead 12 --dropout 0.1 --buffer_size 20 --eval_batch_size 20 --valid_click_path ./data/click_data/test_click.data.gz --save_step 50 --eval_step 50 --n_queries_for_each_gpu 8 --num_candidates 10 --valid_annotate_path ./data/annotate_data/valid_data.txt 

Namespace(_CLS_=0, _MASK_=3, _PAD_=2, _SEP_=1, buffer_size=20, dropout=0.1, emb_dim=768, eval_batch_size=20, eval_step=50, evaluate_model_path='', exp_settings={'method_name': 'NavieAlgorithm', 'n_gpus': 2, 'init_parameters': '', 'lr': 2e-07, 'max_candidate_num': 10, 'selection_bias_cutoff': 10, 'feature_size': 768, 'train_input_hparams': '', 'learning_algorithm_hparams': ''}, finetune_epoch=20, init_parameters='', log_interval=10, lr=2e-07, max_seq_len=128, max_steps=100000, method_name='NavieAlgorithm', n_gpus=2, n_queries_for_each_gpu=8, nhead=12, nlayers=3, ntokens=22000, num_candidates=10, result_path='result.csv', save_step=50, seed=0, test_annotate_path='./data/wsdm_round_1/wsdm_test_1.txt', train_batch_size=160, train_datadir='./data/train_data/', valid_annotate_path='./data/annotate_data/valid_data.txt', valid_click_path='./data/click_data/test_click.data.gz', warmup_steps=4000, weight_decay=0.01)
load annotated data from  ./data/annotate_data/valid_data.txt
/opt/conda/envs/py

### Step 5: Finetuning for Web Search
In this example, we binary human label and train a binary classify model as the final ranking model.

**Set the init_parameters to the path of trained pre-trained model.**

In [ ]:
!cd WSDMCUP_BaiduPLM_Paddle && python -u finetune.py --emb_dim 768 --nlayer 3 --nhead 12 --dropout 0.1 --buffer_size 20 --eval_batch_size 20 --save_step 50 --eval_step 50 --init_parameters '/home/aistudio/data/data186340/save_steps70000_0.98254_1.417266.model' --n_queries_for_each_gpu 8 --num_candidates 10 --ntokens 22872 --valid_annotate_path ./data/annotate_data/finetune.txt --test_annotate_path ./data/annotate_data/valid_data.txt

Namespace(_CLS_=0, _MASK_=3, _PAD_=2, _SEP_=1, buffer_size=20, dropout=0.1, emb_dim=768, eval_batch_size=20, eval_step=50, evaluate_model_path='', exp_settings={'method_name': 'NavieAlgorithm', 'n_gpus': 2, 'init_parameters': '', 'lr': 2e-07, 'max_candidate_num': 10, 'selection_bias_cutoff': 10, 'feature_size': 768, 'train_input_hparams': '', 'learning_algorithm_hparams': ''}, finetune_epoch=20, init_parameters='', log_interval=10, lr=2e-07, max_seq_len=128, max_steps=100000, method_name='NavieAlgorithm', n_gpus=2, n_queries_for_each_gpu=8, nhead=12, nlayers=3, ntokens=22000, num_candidates=10, result_path='result.csv', save_step=50, seed=0, test_annotate_path='./data/annotate_data/valid_data.txt', train_batch_size=160, train_datadir='./data/train_data/', valid_annotate_path='./data/annotate_data/finetune.txt', valid_click_path='./data/click_data/part-00000.gz', warmup_steps=4000, weight_decay=0.01)
Transformer is used for finetune
W1019 17:24:53.993053   621 device_context.cc:447] Ple

### Step 6: Prepare the submission for Pre-training Web Search 
The website for submission: https://aistudio.baidu.com/aistudio/competition/detail/537/0/submit-result?previewCode=aa051a5d-aa48-4be0-83af-4937933a80c2

(a) Download the submission file.

(b) Load the trained model (**Set the init_parameters to the path of trained pre-trained model.**) and generate inference score.

(c) Find the result.csv and submit compressed result.csv.zip to the platform.

In [ ]:
# Download the evaluation set from the AIStudio
!wget "https://bj.bcebos.com/v1/ai-studio-online/2cd7df56206d4e50be0f3a58e53a0236d7eab0cd9ad94f9d82afddba361aa479?responseContentDisposition=attachment%3B%20filename%3Dwsdm_test_1.txt&authorization=bce-auth-v1%2F5cfe9a5e1454405eb2a975c43eace6ec%2F2022-09-14T15%3A30%3A16Z%2F-1%2F%2Feae14185e8787d435e4720811fd707447593c1e6a7b24903daf99684a6391c95" -O wsdm_test_1.txt
!mv wsdm_test_1.txt ./WSDMCUP_BaiduPLM_Paddle/data/
!cd WSDMCUP_BaiduPLM_Paddle && python -u submit.py --emb_dim 768 --nlayer 3 --nhead 12 --init_parameters '' --test_annotate_path "data/wsdm_test_1.txt"  --eval_batch_size 200
# Find the result.csv and submit compressed result.csv.zip to the platform.

--2022-10-19 17:27:07--  https://bj.bcebos.com/v1/ai-studio-online/2cd7df56206d4e50be0f3a58e53a0236d7eab0cd9ad94f9d82afddba361aa479?responseContentDisposition=attachment%3B%20filename%3Dwsdm_test_1.txt&authorization=bce-auth-v1%2F5cfe9a5e1454405eb2a975c43eace6ec%2F2022-09-14T15%3A30%3A16Z%2F-1%2F%2Feae14185e8787d435e4720811fd707447593c1e6a7b24903daf99684a6391c95
Resolving bj.bcebos.com (bj.bcebos.com)... 220.181.33.43, 220.181.33.44, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to bj.bcebos.com (bj.bcebos.com)|220.181.33.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43754463 (42M) [application/octet-stream]
Saving to: ‘wsdm_test_1.txt’

wsdm_test_1.txt     100%[===================>]  41.73M  15.6MB/s    in 2.7s    

2022-10-19 17:27:10 (15.6 MB/s) - ‘wsdm_test_1.txt’ saved [43754463/43754463]

Namespace(_CLS_=0, _MASK_=3, _PAD_=2, _SEP_=1, buffer_size=500000, dropout=0.1, emb_dim=768, eval_batch_size=200, eval_step=500, evaluate_model_path='', exp_settin

### Summary & Discussion 
After submmitting the result, you will receive the score in 5 minutes.

![](https://ai-studio-static-online.cdn.bcebos.com/db304604fc9e424c874d089a52906338bf639d0717f54091b746eadbff316184)


In the pre-training stage, self-supervised learning techniques or few-shot methods are the possible exploration direction for this task.
For the finetuning stage, more advanced pairwise, listwise loss or setrank approach can be employed for boosting performance.
Furthermore, the unigram is also an important information for optimizing the ranking performance.



In [31]:
!cd WSDMCUP_BaiduPLM_Paddle && python -u finetune_test.py --valid_annotate_path ./data/annotate_data/finetune_debug.txt 

        21874, 21874, 21874, 21874, 6164 , 7843 , 21874, 9855 , 16134, 21874,
        21874, 21874, 6164 , 7843 , 21874, 9855 , 16134, 21874, 1    , 2    ,
        2    , 2    , 2    , 2    , 2    , 2    , 2    , 2    , 2    , 2    ,
        2    , 2    , 2    , 2    , 2    , 2    , 2    , 2    ]), Tensor(shape=[128], dtype=int32, place=CUDAPlace(0), stop_gradient=True,
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]), Tensor(shape=[128], dtype=int32, place=CUDAPlace(0), stop_gradient=True,
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,